# Chat Models

LLM ile sohbet etmek için bir chat modelin nasıl kullanılacağını öğreneceğiz.

##### Chat Models vs Language Models
Şimdiye kadar LLM ile iletişim kurmak için bir language model kullanıyorduk. Bir language model, bir kelime dizisindeki bir sonraki kelimeyi tahmin eder. Chat models, ise konuşmalar yapmak için tasarlanmıştır - bir mesaj listesi kabul eder ve bir konuşma yanıtı döndürürler.(they accept a list of messages and return a conversational response.)

Chat models genellikle farklı messages türlerini destekler:

System - LLM'e insan mesajlarına nasıl tepki vereceğini öğretir

Human - kullanıcı tarafından gönderilen mesajlardır

AI - AI'dan gelen yanıtlar


##### Create a Chat Model

Chat model kullanan bir uygulama oluşturacağız.

The application will:

Use a system message to provide instructions

Use a human message to ask a question

Receive an AI response to the question


Yeni bir Python programı oluşturun, gerekli LangChain modüllerini içe aktarın ve sohbet modelini örneklendirin.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage  

chat_llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
# SystemMessage classını kullanarak chat modele talimatlar sağlamak için bir system message oluşturalım.
instructions = SystemMessage(content="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.
""")

In [ ]:
# Bir soru sormak için HumanMessage nesnesini oluşturalım.

question = HumanMessage(content="What is the weather like?")

In [ ]:
# Bu örnekte, sistem mesajını instructions ve insan mesajını question ile ilettik.

response = chat_llm.invoke([
    instructions,
    question
])

print(response.content)

In [ ]:
# The response is an AIMessage object.

AIMessage(content="Dude, the weather is totally gnarly! It's sunny with some epic offshore winds. Perfect conditions for shredding some sick waves!", additional_kwargs={}, example=False)

##### Wrapping in a Chain


Prompt ve chains hakkında öğrendiklerimizi kullanarak yeniden kullanılabilir bir chat modeli chaini oluşturabiliriz. Chat modele bir message list aktarmak yerine, konuşmaya context veren ve sonra soruyu chat modele aktaran bir prompt oluşturabiliriz.


Prompt, system ve human mesajlarının birleştirilmesiyle oluşturulur.

Chain, chat model, prompt ve output parser kullanılarak oluşturulur.

Question, invoke metodunun bir parametresi olarak chat modele iletilir.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

chat_llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        (
            "human", 
            "{question}"
        ),
    ]
)

chat_chain = prompt | chat_llm | StrOutputParser()

response = chat_chain.invoke({"question": "What is the weather like?"})

print(response)


Bir chain oluşturmak, daha sofistike bir chat model oluşturmanın ilk adımıdır. Farklı öğeleri bir çağrıya birleştirmek ve daha karmaşık özellikleri desteklemek için chainleri kullanabiliriz.



#### Giving context


Daha önce, groundingi ve LLM'e nasıl context sağlayabileceğini ve halüsinasyondan kaçınabileceğini öğrendik.

Şu anda, chat modelimiz grounding yapılmadı. Question'a ve LLM'in eğitim verilerine dayanarak (aylar veya yıllar güncel olabilir) yanıt verir.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

chat_llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ( "system", "{context}" ),
        ( "human", "{question}" ),
    ]
)

chat_chain = prompt | chat_llm | StrOutputParser()

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

response = chat_chain.invoke(
    {
        "context": current_weather,
        "question": "What is the weather like on Watergate Bay?",
    }
)

print(response)

Aşağıda tipik bir yanıt var. LLM, daha doğru bir yanıt sağlamak için istemde iletilen bağlamı kullanmıştır.

Dude, the surf at Watergate Bay is pumping! We got some sick 3ft waves rolling in, but unfortunately, we got some onshore winds messing with the lineup. But hey, it's all good, still plenty of stoke to be had out there!


Context sağlamak, RAG'in bir yönüdür. Bu programda, model contextini manuel olarak verdik; Ancak, bir API veya veritabanından real-time bilgi alabilirdik.



#### Giving a Chat Model Memory


Bir sohbet modelinin yardımcı olması için hangi mesajların gönderildiğini ve alındığını hatırlaması gerekir.

Sohbet modelini hatırlama yeteneği olmadan konuşma bağlamına göre hareket edemez.

Örneğin, bir hafıza olmadan konuşma daireler halinde gidebilir:

[user] Hi, my name is Martin

[chat model] Hi, nice to meet you Martin

[user] Do you have a name?

[chat model] I am the chat model. Nice to meet you. What is your name?

#### Conversation Memory - Konuşma belleği

Önceki derste, kullanıcının sorularına yanıt veren bir sohbet modeli oluşturduk. Bu derste, bu programa bir bellek ekleyeceğiz.

Langchain, farklı senaryoları ve depolama çözümlerini destekleyen çeşitli memory componentleri destekler.

Chat model ile aramızdaki konuşma geçmişini geçici olarak depolamak için ChatMessageHistory memory componenti kullanacağız.

#### Add History to the Prompt

As each call to the LLM is stateless, you need to include the chat history in every call to the LLM. You can modify the prompt template to include the chat history as a list of messages using a MessagesPlaceholder object.

LLM'ye yapılan her call stateless olduğundan, her call'da sohbet geçmişini eklememiz gerekir. Prompt template'i, sohbet geçmişini bir MessagePlaceholder nesnesi kullanarak mesajların bir listesi olarak ekleyecek şekilde değiştirebilirsiniz.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

# Chat_history değişkeni konuşma geçmişini içerecektir.